In [179]:
"""

Import all dependencies 

"""

from os import listdir
from mtcnn.mtcnn import MTCNN   ## algorithm for extract faces 
from matplotlib import pyplot
import tensorflow as tf
from PIL import Image
import numpy as np
from os import walk
from os import path
from os import mkdir
from collections import Counter
import pandas as pd
from random import randint


import plotly.express as px
from plotly import offline
from matplotlib import pyplot as plt


import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
from keras import losses




tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)



"""
projectPath = "/home/hanpeng/Projects/ML-FullStack"
detector = MTCNN()
global_count = 0 
def Extract_Face(filename):
    
    global global_count
    global_count = global_count + 1
    if global_count % 1000 == 0:
        print("processing {n}th files".format(n=global_count) )
    try:
        face_image = pyplot.imread(filename)
        detections = detector.detect_faces(face_image)
        if detections == []:
            with open(projectPath+"/extract-face-errors.txt", "w") as text_file:
                text_file.write("no face detected from {filename}".format(filename=filename))
            return

        arr = detections[0]["box"]
        face = face_image[arr[1]:arr[1]+arr[3], arr[0]:arr[0]+arr[2], :]
        img = Image.fromarray(face, 'RGB')
        img.save(projectPath+"/extract-faces/"+filename.split("/")[-1])

    except Exception as e:
        with open(projectPath+"/extract-face-errors.txt", "w") as text_file:
            text_file.write("the file {filename} has error: {error}".format(filename=filename, error=e))


            
image_path = projectPath + "/extract-faces/"
image_files = []
for (dirpath, dirnames, files) in walk(image_path):
    image_files.extend([image_path + f for f in files])
    break
    
"""
print("")

### data source https://susanqq.github.io/UTKFace/   -> for training and validation

In [215]:
class Model_Prep:
    
    def __init__(self, work_dir):
        self.work_dir = work_dir
        self.face_dir = path.join(self.work_dir, "extract-face")
  

    def extract_face(self, image_dir):
        self.count_image = 0
        mkdir(self.face_dir)
        self.detector = MTCNN()
        image_files = []
        for (dirpath, dirnames, files) in walk(image_dir):
            image_files.extend([path.join(image_dir,f) for f in files])
            break
        
       
        for f in image_files:
            self.mtcnn_helper(f)
        
    
    def data_prep(self, percentage_val):
        
        """
        get labels and face files 
        """
        
        for (dirpath, dirnames, files) in walk(self.face_dir):
            face_files = []
            self.labels_int = []
            self.labels_str = []
            face_files = [path.join(dirpath,f) for f in files]
            for f in files:
                face_files.append(path.join(dirpath,f))
                self.labels_int.append(int(f.split("_")[0]))
                self.labels_str.append(f.split("_")[0])
            break
        
        
        """
        pair labels and files 
        
        len(image_files) = labels_str where each label pairs one file  
        """
        

        label_image_dic = {}
        for f,l in zip(face_files, self.labels_int):
            if l not in label_image_dic:
                label_image_dic[l] = [f]
            else:
                label_image_dic[l].append(f)
                
        """
        random split train and val data
        """
        
        percentage_val = percentage_val
        file_val = []
        l_val = []
        file_train = []
        l_train =[]
        for k in label_image_dic.keys():
            l_images = label_image_dic[k]
            num_val = int(len(l_images) * percentage_val) 
            rand_upper = len(l_images) - 1
            if len(l_images) < 10:
                file_val.append(l_images[0])
                l_val.append(k)
                #del label_image_dic[k][rand]
            else:
                for n in range(num_val):
                    index = randint(0,rand_upper)
                    file_val.append(l_images[index])
                    l_val.append(k)
                    del label_image_dic[k][index]
                    rand_upper -= 1
            file_train.extend(label_image_dic[k])
            l_train.extend([k]*len(label_image_dic[k]))
        
        return (file_val, l_val, file_train, l_train)
        
    @staticmethod
    def data_preprocess(file_val, l_val, file_train, l_train, image_dim, batch_size_train,batch_size_val):

        """
        create keras image generator from dataframe
        """
        
        data_gen = ImageDataGenerator(
                        rescale=1.0/255.0,
                        horizontal_flip = True,
                        zoom_range = 0.1,
                        shear_range = 0.1,
                        rotation_range=20,
                        fill_mode = "nearest")
        val_gen = ImageDataGenerator(
                        rescale=1./255)


        df = pd.DataFrame(data={"filename":file_train, "class":l_train})
        df_val = pd.DataFrame(data={"filename":file_val, "class":l_val})
        train_gen = data_gen.flow_from_dataframe(dataframe=df, 
                                                 target_size=image_dim, 
                                                 batch_size=batch_size_train, 
                                                 class_mode = "raw",
                                                 seed=0)
        val_gen = data_gen.flow_from_dataframe(dataframe=df_val, 
                                               target_size=image_dim, 
                                               batch_size=batch_size_val, 
                                               class_mode = "raw",
                                               seed=1)
        
        return (train_gen, val_gen)
        
        
    
    
    def mtcnn_helper(self, image_dir):
        self.count_image += 1 
        image_name = image_dir.split("/")[-1]
        error_log = path.join(self.work_dir, "face-extract-error.txt")
        if self.count_image % 1000 == 0:
            print("processing {n}th files".format(n=self.count_image) )
        
        try:
            face_image = pyplot.imread(image_dir)
            detections = self.detector.detect_faces(face_image)
            if detections == []:
                with open(error_log, "a") as text_file:
                    text_file.write("\n no face detected from {filename}".format(filename=image_name))
                return

            arr = detections[0]["box"]
            face = face_image[arr[1]:arr[1]+arr[3], arr[0]:arr[0]+arr[2], :]
            img = Image.fromarray(face, 'RGB')
            img.save(path.join(self.face_dir,image_name))

        except Exception as e:
            with open(error_log, "a") as text_file:
                text_file.write("\n the file {filename} has error: {error}".format(filename=image_name, error=e))
        

In [214]:
model_prep = Model_Prep("/home/hanpeng/Desktop/model-training-test")
(file_val, l_val, file_train, l_train) = model_prep.data_prep(0.1)
(train_gen, val_gen) = test.data_preprocess(file_val, l_val, file_train, l_train, (150,150),32,16)

Found 21259 validated image filenames.
Found 2320 validated image filenames.


### our image data is not even close to uniform distribution, this will cause some biases when we train our networks



In [236]:
"""
some visualization of our image data
"""


l_dict = Counter(model_prep.labels_str)
fig = px.bar(x = list(l_dict.keys()), y = list(l_dict.values()))
fig.update_layout(
    title="number of images against each age",
    xaxis_title="age",
    yaxis_title="# of images",
)

fig.show()

In [222]:
class Model:
    @staticmethod
    def train_model(image_dims,n_class,model=None, **kwargs):
        if model!=None:
            """
            train model from check point
            """
            
            model.compile(loss='mse',
                        optimizer=optimizers.Adam(lr=0.0007),
                        metrics=["MAE"])
            model_output = model.fit_generator(
                                kwargs["train_data"],
                                steps_per_epoch=kwargs["steps_per_epoch"],
                                epochs=kwargs["epochs"],
                                validation_data=kwargs["val_data"],
                                validation_steps=kwargs["validation_steps"])

            return (model, model_output)
        

        model = models.Sequential()
        #model.add(conv_base)
        model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=image_dims))
        model.add(layers.MaxPooling2D((3, 3)))
        model.add(layers.Conv2D(64, (3, 3), activation="relu"))
        model.add(layers.MaxPooling2D((3, 3)))
        model.add(layers.Conv2D(128, (3, 3), activation="relu"))
        model.add(layers.MaxPooling2D((3, 3)))
        model.add(layers.Conv2D(256, (3, 3), activation="relu"))
        model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Flatten())
        model.add(layers.Dense(1,activation='linear')) ## regression mode


        model.compile(loss='mse',
                        optimizer=optimizers.Adam(lr=0.001),
                        metrics=["MAE"])

        model_output = model.fit_generator(
            kwargs["train_data"],
            steps_per_epoch=kwargs["steps_per_epoch"],
            epochs=kwargs["epochs"],
            validation_data=kwargs["val_data"],
            validation_steps=kwargs["validation_steps"])

        return (model,model_output)

In [223]:
model = Model()
(model_details, model_output) = model.train_model((150,150,3),
                                                  n_class=None, 
                                                  train_data=train_gen,
                                                  val_data=val_gen,
                                                  steps_per_epoch=30,
                                                  epochs=100,
                                                  validation_steps=10)

Epoch 1/100
30/30 [==============================] - 8s 270ms/step - loss: 684.3149 - MAE: 20.6896 - val_loss: 479.7855 - val_MAE: 17.8313
Epoch 2/100
30/30 [==============================] - 8s 271ms/step - loss: 443.8461 - MAE: 16.1536 - val_loss: 366.5098 - val_MAE: 15.0454
Epoch 3/100
30/30 [==============================] - 8s 272ms/step - loss: 452.2240 - MAE: 16.5382 - val_loss: 506.8035 - val_MAE: 18.1861
Epoch 4/100
30/30 [==============================] - 8s 282ms/step - loss: 439.7158 - MAE: 16.0455 - val_loss: 376.9959 - val_MAE: 14.5672
Epoch 5/100
30/30 [==============================] - 9s 284ms/step - loss: 442.1583 - MAE: 16.2241 - val_loss: 532.6597 - val_MAE: 17.7727
Epoch 6/100
30/30 [==============================] - 8s 283ms/step - loss: 432.4706 - MAE: 15.7428 - val_loss: 438.9462 - val_MAE: 16.8861
Epoch 7/100
30/30 [==============================] - 8s 277ms/step - loss: 431.2881 - MAE: 16.0335 - val_loss: 465.7508 - val_MAE: 16.7738
Epoch 8/100
30/30 [========

30/30 [==============================] - 8s 276ms/step - loss: 247.8058 - MAE: 11.7753 - val_loss: 254.1042 - val_MAE: 12.8311
Epoch 60/100
30/30 [==============================] - 8s 278ms/step - loss: 259.6040 - MAE: 12.3402 - val_loss: 258.3065 - val_MAE: 11.9531
Epoch 61/100
30/30 [==============================] - 8s 276ms/step - loss: 220.9527 - MAE: 11.2687 - val_loss: 302.4932 - val_MAE: 12.7342
Epoch 62/100
30/30 [==============================] - 8s 283ms/step - loss: 268.3285 - MAE: 12.4102 - val_loss: 251.0111 - val_MAE: 12.2902
Epoch 63/100
30/30 [==============================] - 9s 288ms/step - loss: 244.0324 - MAE: 12.0104 - val_loss: 282.0891 - val_MAE: 12.4553
Epoch 64/100
30/30 [==============================] - 8s 271ms/step - loss: 243.9968 - MAE: 11.9610 - val_loss: 240.0552 - val_MAE: 12.2267
Epoch 65/100
30/30 [==============================] - 8s 274ms/step - loss: 272.8280 - MAE: 12.5220 - val_loss: 284.3343 - val_MAE: 12.5941
Epoch 66/100
30/30 [=============

### you can train model much longer to reduce the mean absolute error, on my end I cound reduce it to 3
### let's visualize our loss over time

In [71]:
pred = model.predict(test_data,verbose=1)

13731/13731 [==============================] - 64s 5ms/step


In [246]:
train_mae = {'type' : 'scatter', 'mode' : 'lines', 
            'x' : model_output.epoch, 'y' : model_output.history["MAE"], 'line': {'shape': 'spline', 'smoothing': 1.3},
            }

val_mae = {'type' : 'scatter', 'mode' : 'lines', 
'x' : model_output.epoch, 'y' : model_output.history["val_MAE"], 'line': {'shape': 'spline', 'smoothing': 1.3}}

train_loss = {'type' : 'scatter', 'mode' : 'lines', 
'x' : model_output.epoch, 'y' : model_output.history["loss"], 'line': {'shape': 'spline', 'smoothing': 1.3}}


val_loss = {'type' : 'scatter', 'mode' : 'lines', 
'x' : model_output.epoch, 'y' : model_output.history["val_loss"], 'line': {'shape': 'spline', 'smoothing': 1.3}}

offline.iplot([train_mae,val_mae])
offline.iplot([train_loss,val_loss])